In [1]:
import os
os.environ['JAX_CHECK_TRACER_LEAKS'] = '1'

In [2]:
import sys
sys.path.append('/home/jaxmao/jaxmao_branches/JaxMao/')

import jax
from jaxmao.modules import Module
from jaxmao.layers import Dense, BatchNorm
from jaxmao.losses import CategoricalCrossEntropy
from jaxmao.optimizers import GradientDescent
from jaxmao.utils_struct import _check_dict_ids

import numpy as np

from sklearn.datasets import load_digits

In [3]:
images, targets = load_digits(return_X_y=True)

images = images / images.max()
targets_enc = jax.nn.one_hot(targets, num_classes=10)

I0000 00:00:1698329373.362794  115546 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


In [6]:
class Classifier(Module):
    def __init__(self):
        super().__init__()
        self.dense1 = Dense(64, 32)
        self.dense2 = Dense(32, 10)
        self.bn1 = BatchNorm(32)
        self.bn2 = BatchNorm(10)
        self.bn1.training = True
        self.bn2.training = True
        
    def __call__(self, x):
        x = self.dense1(x)
        # x = self.bn1(x)
        # x = jax.nn.relu(x)
        
        x = self.dense2(x)
        # x = self.bn2(x)
        # x = jax.nn.softmax(x)
        return x

seed = 42
key = jax.random.key(seed)

clf = Classifier()
clf.init_params(key)

{'dense1': {'weights': Array([[ 0.02117668, -0.22214548, -0.1634809 , ...,  0.02727365,
          -0.09970619, -0.13141589],
         [ 0.22423503, -0.31929344, -0.05311856, ...,  0.12617505,
          -0.05755344, -0.21855643],
         [ 0.01177022, -0.06092661, -0.0791641 , ..., -0.08742873,
           0.15001932, -0.03505825],
         ...,
         [-0.1453791 , -0.01812402,  0.28515035, ..., -0.08504849,
           0.1471512 , -0.2206304 ],
         [-0.02522955, -0.00388724,  0.03972574, ...,  0.20668949,
           0.04916492, -0.22023007],
         [ 0.17874895,  0.0127775 ,  0.17311396, ...,  0.00680099,
          -0.1472737 , -0.14495818]], dtype=float32),
  'biases': Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],      dtype=float32)},
 'dense2': {'weights': Array([[ 0.09024656,  0.11920545, -0.17902997, -0.15849029,  0.05521125,
          -0.37190118,  0.696331  , -0.41349685, 

training loop

In [7]:
loss_fn = CategoricalCrossEntropy(reduce_fn='mean_over_batch_size')
optimizer = GradientDescent(lr=0.01, params=clf.params)

# @jax.jit
def loss_fn_wrapped(params, x, y):
    y_pred = clf.apply(params, x)
    loss = loss_fn(y_pred, y)
    return loss

loss_and_grad = jax.value_and_grad(loss_fn_wrapped, argnums=0, has_aux=False)
loss, gradients = loss_and_grad(clf.params, images, targets_enc)

with jax.check_tracer_leaks() as T:
    EPOCHS = 5
    BATCH_SIZE = 128    
    NUM_BATCHES = len(images) // BATCH_SIZE
    for epoch in range(EPOCHS):
        total_losses = 0.0
        for n in range(BATCH_SIZE): 
            loss, gradients = loss_and_grad(clf.params, images, targets_enc)
            new_params, optimizer.state = optimizer(clf.params, gradients, optimizer.state)
            # clf.update_params(new_params)
            # clf.update_state()
            total_losses += loss
        print('epoch: {} - avg_loss: {} {}'.format(epoch+1, total_losses/NUM_BATCHES, T))

Exception: Leaked trace MainTrace(2,JVPTrace). Leaked tracer(s):

Traced<ConcreteArray([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],      dtype=float32)
  tangent = Traced<ShapedArray(float32[32])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[32]), None)
    recipe = LambdaBinding()
<JVPTracer 140492927902368> is referred to by <dict 140492926503040>['beta']
<dict 140492926503040> is referred to by <BatchNorm 140497715956496>.params
<BatchNorm 140497715956496> is referred to by <Classifier 140497878812880>.bn1
<Classifier 140497878812880> is referred to by __main__.clf

Traced<ConcreteArray([1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.], dtype=float32)>with<JVPTrace(

ids are consistent. There are no copies of same items.

In [ ]:
print('params dense1: ', _check_dict_ids(clf.params['dense1'], clf.layers['dense1'].params))
print('params dense2: ', _check_dict_ids(clf.params['dense2'], clf.layers['dense2'].params))

print('state dense1: ', _check_dict_ids(clf.state['dense1'], clf.layers['dense1'].state))
print('state dense2: ', _check_dict_ids(clf.state['dense2'], clf.layers['dense2'].state))

params dense1:  False
params dense2:  False
state dense1:  True
state dense2:  True


result

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

y_pred = clf(images)
y_pred2 = clf.apply(clf.params, images)
print('Equivalent: ', (y_pred==y_pred2).all())

y_pred = y_pred.argmax(axis=1)
accuracy = accuracy_score(targets, y_pred)
precision = precision_score(targets, y_pred, average='macro')
recall = recall_score(targets, y_pred, average='macro')
print('Accuracy : {:<.6f}'.format(accuracy))
print('Precision: {:<.6f}'.format(precision))
print('Recall   : {:<.6f}'.format(recall))

Equivalent:  False
Accuracy : 0.968280
Precision: 0.968663
Recall   : 0.968199
